<br></br>
# **gensim | word2vec**
<br></br>
## **1 데이터 전처리**

In [ ]:
# 독일 퀘르버 재단 연설문 : 베를린 선언
f     = open('./data/베를린선언.txt', 'r')
texts = f.read()
f.close()

texts = texts.replace('\n\n', '\n')
texts = texts.replace('\n\n', '\n')
texts[:500]

In [ ]:
# 텍스트를 한 줄씩 문법 tag를 추가한다
from konlpy.tag import Okt
twitter    = Okt()

token_sent = texts.split('\n')
token_sent[:5]

In [ ]:
%%time
results_sent = []
for token in token_sent:
    twitter_token = twitter.pos(token, norm=True, stem=True)
    results = [ word[0]   
               for word in twitter_token    # 어미/조사/구두점 제외
               if not word[1] in ["Eomi", "Josa", "Punctuation"] ]
    rl = (" ".join(results)).strip()
    results_sent.append(rl)
print(results_sent[:5])

In [ ]:
texts_file = './data/Berlin.tagged'
with open(texts_file, 'w', encoding='utf-8') as file:
    file.write("\n".join(results_sent))

In [ ]:
! cat ./data/Berlin.tagged | head -n 5

<br></br>
## **2 Word 2 vec 객체 만들기**
gensim

In [ ]:
# ! pip3 install gensim

In [ ]:
%%time
texts_file = './data/Berlin.tagged'

from gensim.models import word2vec
data  = word2vec.LineSentence(texts_file)
model = word2vec.Word2Vec(data, size=200, window=2, hs=1, min_count=2, sg=1)
model.save("./data/Berlin.model")
print("model saved.")

<br></br>
## **3 저장된 객체 활용**
gensim

In [ ]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('./data/Berlin.model')
model.wv.most_similar(positive=['한반도'])

In [ ]:
model.wv.most_similar(positive=['통일'])

In [ ]:
model.wv.most_similar(positive=['정치'])

In [ ]:
model.wv.most_similar(positive=['대한민국', '한반도'], 
                      negative=['정치'])

<br></br>
## **04 Visulaization**
gensim

In [ ]:
# model.wv.vocab : { word: object of numeric vector }
vocab  = list(model.wv.vocab)
X      = model[vocab]
X.shape

In [ ]:
from sklearn.manifold import TSNE
tsne   = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)

In [ ]:
import pandas as pd
df = pd.DataFrame(X_tsne, 
                  index = vocab, 
                  columns=['x', 'y'])
df.head()

In [ ]:
%matplotlib inline
from matplotlib import rc
rc('font', family='NanumGothic')

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,12))
ax  = fig.add_subplot(1, 1, 1)
ax.scatter(df['x'], df['y'])
for word, pos in df.iterrows():
    ax.annotate(word, pos)
plt.grid(True)